In [4]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

# Pair Programming ETL I

1. Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. NOTA: nos piden que el código debe estar en una función.

In [ ]:
def llamada_API_peninsula(año_inicio, año_final):
    df_peninsula = pd.DataFrame()
    
    for i in range(año_inicio, (año_final + 1)):
            
            url1 = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=peninsular&geo_ids=8741'
            response1 = requests.get(url=url1)
            if response1.status_code != 200:
                print(response1.reason, response1.status_code)
                break
            
            for x in range(len(response1.json()['included'])):
                df = pd.DataFrame(response1.json()['included'][x]['attributes']['values'])
                df['tipo_energia'] = response1.json()['included'][x]['attributes']['title']
                
                df_peninsula = pd.concat([df_peninsula, df], axis=0)
                
    return df_peninsula

In [ ]:
df_peninsula = llamada_API_peninsula(2011, 2022)

In [ ]:
df_peninsula.sample()

In [40]:
df_peninsula.to_csv('datos/energia_año_peninsula.csv')

2. Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [3]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [27]:
def llamar_api_ccaa(ano_inicio, ano_final, **dicc_ccaa,):
    
    df_final = pd.DataFrame()
    
    for i in range(ano_inicio, (ano_final + 1)):
        
        for k, v in dicc_ccaa.items():

            url1 = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}'
            response1 = requests.get(url=url1)
            if response1.status_code != 200:
                print(response1.reason, response1.status_code)
                break
            
            for x in range(len(response1.json()['included'])):
                df = pd.DataFrame(response1.json()['included'][x]['attributes']['values'])
                df['tipo_energia'] = response1.json()['included'][x]['attributes']['title']
                df['comunidad'] = k
                df['cod_comunidad'] = v
                
                df_final = pd.concat([df_final, df], axis=0)
                 
    return df_final

In [28]:
energia_comunidades = llamar_api_ccaa(2011, 2023, **cod_comunidades)

In [29]:
energia_comunidades.sample()

,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad
0,1.178491e+06,0.163188,2011-01-01T00:00:00.000+01:00,Renovable,País Vasco,10


3. Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs

In [36]:
df_final = pd.concat([df_peninsula, energia_comunidades], join='outer', axis=0)
df_final

,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad
0,8.583292e+07,0.324714,2011-01-01T00:00:00.000+01:00,Renovable,NaN,NaN
0,1.785007e+08,0.675286,2011-01-01T00:00:00.000+01:00,No renovable,NaN,NaN
0,8.446393e+07,0.314513,2012-01-01T00:00:00.000+01:00,Renovable,NaN,NaN
0,1.840909e+08,0.685487,2012-01-01T00:00:00.000+01:00,No renovable,NaN,NaN
0,1.088507e+08,0.418855,2013-01-01T00:00:00.000+01:00,Renovable,NaN,NaN
...,...,...,...,...,...,...
0,2.149877e+06,0.809786,2023-01-01T00:00:00.000+01:00,No renovable,Canarias,8742.0
0,1.969101e+05,0.326280,2023-01-01T00:00:00.000+01:00,Renovable,Región de Murcia,21.0
0,4.065906e+05,0.673720,2023-01-01T00:00:00.000+01:00,No renovable,Región de Murcia,21.0
0,1.312421e+05,0.499506,2023-01-01T00:00:00.000+01:00,Renovable,La Rioja,20.0


In [75]:
energia_comunidades.to_csv('datos/energia_comunidades.csv')